In [1]:
import openml
from mlaut.data import Data
import pandas as pd
from sklearn import preprocessing
import os

/media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
apikey = 'd2b1d13981d4abfb22895337baca924c'
openml.config.apikey = apikey
openml.config.set_cache_directory(os.path.expanduser('~/.openml/cache'))

In [12]:
classification_tasks = openml.tasks.list_tasks(task_type_id=1)
regression_tasks = openml.tasks.list_tasks(task_type_id=2)

In [3]:
all_datasets = openml.datasets.list_datasets()

In [35]:
all_datasets.keys()

dict_keys([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 70, 71, 72, 73, 74, 75, 76, 77, 78, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 171, 172, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 285, 287, 293, 294, 296, 298, 299, 300, 301, 307, 308, 310, 

In [44]:
all_datasets[396]

{'MajorityClassSize': 943,
 'MaxNominalAttDistinctValues': -1,
 'MinorityClassSize': 273,
 'NumberOfClasses': 6,
 'NumberOfFeatures': 13196,
 'NumberOfInstances': 3204,
 'NumberOfInstancesWithMissingValues': 0,
 'NumberOfMissingValues': 0,
 'NumberOfNumericFeatures': 13195,
 'NumberOfSymbolicFeatures': 1,
 'did': 396,
 'format': 'Sparse_ARFF',
 'name': 'la1s.wc',
 'status': 'active'}

In [45]:
dts=openml.datasets.get_dataset(396)

In [41]:
dts.get_data(return_attribute_names=True)

[array([[nan,  0.,  1., ...,  0., nan,  2.],
        [nan,  0.,  0., ...,  0., nan,  2.],
        [nan,  0.,  0., ...,  0., nan,  2.],
        ...,
        [nan,  0.,  7., ...,  0., nan,  1.],
        [nan,  0.,  1., ...,  0., nan,  5.],
        [nan,  0.,  1., ...,  1., nan,  5.]], dtype=float32),
 ['family',
  'product-type',
  'steel',
  'carbon',
  'hardness',
  'temper_rolling',
  'condition',
  'formability',
  'strength',
  'non-ageing',
  'surface-finish',
  'surface-quality',
  'enamelability',
  'bc',
  'bf',
  'bt',
  'bw%2Fme',
  'bl',
  'm',
  'chrom',
  'phos',
  'cbond',
  'marvi',
  'exptl',
  'ferro',
  'corr',
  'blue%2Fbright%2Fvarn%2Fclean',
  'lustre',
  'jurofm',
  's',
  'p',
  'shape',
  'thick',
  'width',
  'len',
  'oil',
  'bore',
  'packing',
  'class']]

In [32]:
dts.__dict__['qualities']

{'AutoCorrelation': '0.35476718403547675',
 'CfsSubsetEval_DecisionStumpAUC': '0.7653966063675891',
 'CfsSubsetEval_DecisionStumpErrRate': '0.30973451327433627',
 'CfsSubsetEval_DecisionStumpKappa': '0.5175505474063004',
 'CfsSubsetEval_NaiveBayesAUC': '0.7653966063675891',
 'CfsSubsetEval_NaiveBayesErrRate': '0.30973451327433627',
 'CfsSubsetEval_NaiveBayesKappa': '0.5175505474063004',
 'CfsSubsetEval_kNN1NAUC': '0.7653966063675891',
 'CfsSubsetEval_kNN1NErrRate': '0.30973451327433627',
 'CfsSubsetEval_kNN1NKappa': '0.5175505474063004',
 'ClassEntropy': '2.4013863002194658',
 'DecisionStumpAUC': '0.6015576834475163',
 'DecisionStumpErrRate': '0.4336283185840708',
 'DecisionStumpKappa': '0.1875349639126567',
 'Dimensionality': '0.6194690265486725',
 'EquivalentNumberOfAtts': '150.93464090068497',
 'J48.00001.AUC': '0.7383415107504332',
 'J48.00001.ErrRate': '0.34292035398230086',
 'J48.00001.Kappa': '0.47954893250330566',
 'J48.0001.AUC': '0.7383415107504332',
 'J48.0001.ErrRate': '0.3

In [ ]:
tasks = openml.tasks.list_tasks(task_type_id=1)
#filter = tasks.task_type == 'Supervised Classification'

In [ ]:
for t in tasks:
    print(t)

In [ ]:
openml.datasets.get_dataset(238)

In [ ]:
X, names = dataset.get_data(return_attribute_names=True)

In [ ]:
data = Data()
input_io = data.open_hdf5('data/openml.h5', mode='a')

In [ ]:
for id in classification_tasks.keys():
    try:
        dataset = openml.datasets.get_dataset(id)
        X, names = dataset.get_data(return_attribute_names=True)

        #ignore datasets with empty values 
        num_missing_values = float(dataset.__dict__['qualities']['NumberOfMissingValues'])
        if num_missing_values > 0:
            print(f'skipping dataset {id}. Missing values')
            continue

        #ignore too big datasets
        if classification_tasks[id]['NumberOfInstances'] > 10000:
            print('skipping dataset {id}. It is too big')
            continue

        metadata = {
            'class_name': dataset.__dict__['default_target_attribute'],
            'source': 'OpenML',
            'dataset_name':dataset.__dict__['name']
        }
        class_name_index = names.index(metadata['class_name'])

        #Normalize the data
#         scaler = preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True)
#         scaler.fit(X)
#         x_transformed  = scaler.transform(X)
#         x_transformed[:,class_name_index] = X[:, class_name_index]

        #Convert to DataFrame
        result = pd.DataFrame(x_transformed)
        result.columns=names
        result[metadata['class_name']] =  result[metadata['class_name']].astype(int)

        #save to hdf5
        input_io.save_pandas_dataset(dataset=result, save_loc='/openml', metadata=metadata)
        print(f'dataset {id} saved.')

    except:
        print(f'Cannot save dataset {id}')